In [1]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [2]:
def Get_Dectection_Model_Not_Export(pipeline_config, checkpoint_path):
    tf.keras.backend.clear_session()
    
    configs = config_util.get_configs_from_pipeline_file(pipeline_config)
    model_config = configs['model']
    # model_config.ssd.freeze_batchnorm = True
    detection_model = model_builder.build(
          model_config=model_config, is_training=True)
    
    fake_box_predictor = tf.compat.v2.train.Checkpoint(
        _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
        _prediction_heads=detection_model._box_predictor._prediction_heads,
        _box_prediction_head=detection_model._box_predictor._box_prediction_head,
        )
    
    fake_model = tf.compat.v2.train.Checkpoint(
              _feature_extractor=detection_model._feature_extractor,
              _box_predictor=fake_box_predictor)
    ckpt = tf.compat.v2.train.Checkpoint(model=fake_model)
    ckpt_manager = tf.train.CheckpointManager(ckpt, directory=checkpoint_path, max_to_keep=5)
    print(ckpt_manager.latest_checkpoint)
    ckpt.restore(ckpt_manager.latest_checkpoint)
    # ckpt.restore(checkpoint_path).expect_partial()
    
    # Run model through a dummy image so that variables are created
    image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
    prediction_dict = detection_model.predict(image, shapes)
    _ = detection_model.postprocess(prediction_dict, shapes)
    print('Weights restored!')
    return detection_model

In [3]:
pipeline_config = r"C:\Users\sky66\Downloads\models\research\ckpt\pipeline.config"
checkpoint_path = r"C:\Users\sky66\Downloads\models\research\ckpt"
detection_model = Get_Dectection_Model_Not_Export(pipeline_config, checkpoint_path)

@tf.function(input_signature=[tf.TensorSpec(shape=[None,640,640,3], dtype=tf.float32)])
def detect(input_tensor):
    # Note that the first frame will trigger tracing of the tf.function, which will
    # take some time, after which inference should be fast. 
    """Run detection on an input image.
    
    Args:
      input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
        Note that height and width can be anything since the image will be
        immediately resized according to the needs of the model within this
        function.
    
    Returns:
      A dict containing 3 Tensors (`detection_boxes`, `detection_classes`,
        and `detection_scores`).
    """
    preprocessed_image, shapes = detection_model.preprocess(input_tensor)
    prediction_dict = detection_model.predict(preprocessed_image, shapes)
    return detection_model.postprocess(prediction_dict, shapes)

C:\Users\sky66\Downloads\models\research\ckpt\ckpt-1
Weights restored!


In [4]:
tf.saved_model.save(
    detection_model , 'detection_model',
    signatures={
      'detection': detect.get_concrete_function()
    })

Instructions for updating:
Use fn_output_signature instead


INFO:tensorflow:Assets written to: detection_model\assets


INFO:tensorflow:Assets written to: detection_model\assets


# Use 7zip to compose the model folder,
# create "detection model for google drive" folder

In [5]:
# 為了防止 colab gdown 下載檔案時因檔案過大而出現無法掃描病毒，造成下載失敗。
def export_for_google_drive(file, target_folder, size = 10*1024*1024):
    target_folder_files = target_folder+"\\%d"
    
    # Make target folder is empty.
    for e in os.listdir(target_folder):
        f = target_folder+"\\"+e
        os.remove(f)
    
    # Read source file, then write them to files by numbers.
    with open(file, "rb") as f:
        # 每個檔案大小 10MB
        # size = 10*1024*1024 # 單位 bytes
        i = 1
        while True:
            tmp = f.read(size)
            
            if len(tmp)==0 :
                break
            
            with open(target_folder_files%i, "wb") as gf:
                gf.write(tmp)
                
            i += 1

In [9]:
try:
    os.mkdir("detection model for google drive")
    os.mkdir("UNet model for google drive")
except:
    pass

In [10]:
# Limit 25MB

In [11]:
file = "detection_model.zip"
target = "detection model for google drive"
export_for_google_drive(file, target, 20*1024*1024)

In [12]:
file = r"C:\Users\sky66\Desktop\my_model.zip"
target = "UNet model for google drive"
export_for_google_drive(file, target, 20*1024*1024)

In [13]:
def export_from_split_file(source_folder, target_file):
    # create a empty file
    with open(target_file, "wb") as gf:
        gf.write(b"")
    
    # write all buffers to a new file
    export = sorted([int(i) for i in os.listdir(source_folder)])
    # For each source file, we read it, apppending to new file.
    with open(target_file, "ab") as f:
        
        for _ in export:
            tmp = source_folder+"\\"+str(_)
            
            with open(tmp, "rb") as f1:
                tmp = f1.read(-1)
            
            f.write(tmp)

In [18]:
# new_file = "new_detection_model.zip"
# folder = "detection model for google drive"
# export_from_split_file(folder, new_file)

In [13]:
#!gdown --no-cookies --folder "ID"